# Fake News Classificer Using Bidirectional LSTM
#### Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv

## Import all Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer ##stemming purpose
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report


2.12.0


## Import Datasets & Visualizing

In [2]:
#df=pd.read_csv('FakeNewsData/train.csv',engine='python',error_bad_lines=False)
df=pd.read_csv('FakeNewsData/train.csv')

In [3]:
print(df.head(),'\n',df.shape)

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1   
 (20800, 5)


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df=df.dropna()

In [6]:
print(df.head(),'\n',df.shape)

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1   
 (18285, 5)


## Get the Independent & Dependent Features

In [7]:
X=df.drop('label',axis=1)
y=df['label']

In [8]:
print(X.shape,'\n',y.shape)

(18285, 4) 
 (18285,)


In [9]:
## Check whether dataset is balanced or not
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [10]:
## vocabulary size
voc_size=5000

## Dataset Preprocessing

In [11]:
messages=X.copy(deep=True)

In [12]:
print(messages.head(10),'\n',messages['title'][1])

    id                                              title  \
0    0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1    1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2    2                  Why the Truth Might Get You Fired   
3    3  15 Civilians Killed In Single US Airstrike Hav...   
4    4  Iranian woman jailed for fictional unpublished...   
5    5  Jackie Mason: Hollywood Would Love Trump if He...   
7    7  Benoît Hamon Wins French Socialist Party’s Pre...   
9    9  A Back-Channel Plan for Ukraine and Russia, Co...   
10  10  Obama’s Organizing for Action Partners with So...   
11  11  BBC Comedy Sketch "Real Housewives of ISIS" Ca...   

                          author  \
0                  Darrell Lucus   
1                Daniel J. Flynn   
2             Consortiumnews.com   
3                Jessica Purkiss   
4                 Howard Portnoy   
5                Daniel Nussbaum   
7                Alissa J. Rubin   
9   Megan Twohey and Scott Shane   
10  

In [13]:
messages.reset_index(inplace=True)

In [14]:
print(messages.head(10),'\n',messages['title'][1])

   index  id                                              title  \
0      0   0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1      1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2      2   2                  Why the Truth Might Get You Fired   
3      3   3  15 Civilians Killed In Single US Airstrike Hav...   
4      4   4  Iranian woman jailed for fictional unpublished...   
5      5   5  Jackie Mason: Hollywood Would Love Trump if He...   
6      7   7  Benoît Hamon Wins French Socialist Party’s Pre...   
7      9   9  A Back-Channel Plan for Ukraine and Russia, Co...   
8     10  10  Obama’s Organizing for Action Partners with So...   
9     11  11  BBC Comedy Sketch "Real Housewives of ISIS" Ca...   

                         author  \
0                 Darrell Lucus   
1               Daniel J. Flynn   
2            Consortiumnews.com   
3               Jessica Purkiss   
4                Howard Portnoy   
5               Daniel Nussbaum   
6               A

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

## Onehot Representation

In [18]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[359, 696, 680, 1853, 1247, 3856, 471, 2250, 2174, 1304],
 [926, 389, 3583, 428, 60, 3733, 3093],
 [349, 3498, 1776, 2981],
 [204, 2660, 4876, 3562, 3069, 4001],
 [431, 60, 825, 4718, 1167, 1366, 60, 1038, 3592, 1460],
 [2388,
  1700,
  2651,
  1145,
  2427,
  1010,
  4387,
  2964,
  2411,
  3363,
  250,
  2777,
  1162,
  242,
  3093],
 [4850, 3990, 1805, 1104, 2735, 460, 2027, 1414, 4155, 3840, 4974],
 [4929, 207, 4666, 3333, 1039, 4395, 1010, 2094, 4155, 3840, 4974],
 [3814, 4797, 4893, 765, 723, 3260, 3518, 4471, 1010, 1861],
 [4642, 196, 4903, 3208, 1290, 3982, 4004, 1487],
 [4407, 3512, 3808, 2806, 4830, 3250, 3488, 4631, 1507, 3996, 3288],
 [3562, 740, 1247, 3260, 1010, 1039],
 [277, 4731, 521, 4452, 4299, 4028, 2282, 3012, 3006],
 [3462, 4365, 3116, 2647, 344, 767, 306, 4155, 3840, 4974],
 [3358, 923, 4347, 748, 3872, 4155, 3840, 4974],
 [4452, 2358, 465, 3968, 876, 1190, 3439, 488, 1733, 1923],
 [1945, 472, 389],
 [570, 3694, 4287, 2610, 1010, 1244, 4835, 3093],
 [1330, 1722, 

In [19]:
print(corpus[1],'\n',onehot_repr[1])

flynn hillari clinton big woman campu breitbart 
 [926, 389, 3583, 428, 60, 3733, 3093]


## Embedding Representation

In [20]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2250 2174 1304]
 [   0    0    0 ...   60 3733 3093]
 [   0    0    0 ... 3498 1776 2981]
 ...
 [   0    0    0 ... 4155 3840 4974]
 [   0    0    0 ... 3568 4217 2117]
 [   0    0    0 ... 1006  611  140]]


In [21]:
print(embedded_docs[1],'\n', embedded_docs[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0  926
  389 3583  428   60 3733 3093] 
 [   0    0    0    0    0    0    0    0    0    0  359  696  680 1853
 1247 3856  471 2250 2174 1304]


## Creating model

In [22]:

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs),y.shape

(18285, (18285,))

In [24]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

## Train Test Split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [26]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 13s 25ms/step - loss: 0.2616 - accuracy: 0.8825 - val_loss: 0.2017 - val_accuracy: 0.9087
Epoch 2/10
383/383 [==============================] - 9s 23ms/step - loss: 0.1326 - accuracy: 0.9495 - val_loss: 0.2097 - val_accuracy: 0.9140
Epoch 3/10
383/383 [==============================] - 8s 22ms/step - loss: 0.0843 - accuracy: 0.9686 - val_loss: 0.2507 - val_accuracy: 0.9135
Epoch 4/10
383/383 [==============================] - 8s 22ms/step - loss: 0.0506 - accuracy: 0.9822 - val_loss: 0.2797 - val_accuracy: 0.9160
Epoch 5/10
383/383 [==============================] - 9s 22ms/step - loss: 0.0292 - accuracy: 0.9905 - val_loss: 0.3566 - val_accuracy: 0.9140
Epoch 6/10
383/383 [==============================] - 9s 22ms/step - loss: 0.0181 - accuracy: 0.9940 - val_loss: 0.4062 - val_accuracy: 0.9143
Epoch 7/10
383/383 [==============================] - 9s 23ms/step - loss: 0.0114 - accuracy: 0.9967 - val_loss: 0.5017 - val_accuracy: 0.904

## Performance Metrics And Accuracy

In [27]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 6ms/step


In [28]:
y_pred=np.where(y_pred>=0.5,1,0)

In [29]:
confusion_matrix(y_test,y_pred)

array([[3158,  261],
       [ 303, 2313]], dtype=int64)

In [30]:
print(accuracy_score(y_test,y_pred))

0.9065451532725767


In [31]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      3419
           1       0.90      0.88      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.90      0.90      6035
weighted avg       0.91      0.91      0.91      6035

